<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/AskAPatient/Experiment%20(Change%20testing%20dataset)/Codes/ask_pubmed_tweet_overlap_40.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
if os.path.exists('medical-concept-normalization'):
  print('folder exist')
else:
  os.mkdir(base_dir)

In [0]:
if os.path.exists('medical-concept-normalization/data_collection'):
  print('folder exist')
else:
  os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [7]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [8]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# datasets

train_csv = 'https://raw.githubusercontent.com/unt-iialab/medical-concept-normalization/master/AskAPatient/Experiment%20(Change%20testing%20dataset)/Training%20set/AskAPatient.fold-0.train.csv'
valid_csv = 'https://raw.githubusercontent.com/unt-iialab/medical-concept-normalization/master/AskAPatient/Experiment%20(Change%20testing%20dataset)/Validation%20set/AskAPatient.fold-0.validation.csv'
test_csv = 'https://raw.githubusercontent.com/unt-iialab/medical-concept-normalization/master/AskAPatient/Experiment%20(Change%20testing%20dataset)/Testing%20set/AskAPatient_overlap_40.csv'
fine_tuning_data = 'fine_tuning_language_model_pubmed_bigtweet_mixed.csv'

In [12]:
# training data set
train = pd.read_csv(train_csv, header=None)
print(train.head())

# validating data set
valid = pd.read_csv(valid_csv,header=None)
# valid.head()

# testing data set
test = pd.read_csv(test_csv,header=None)
# test.head()

   0                1
0  0             Fear
1  0             Fear
2  0             Fear
3  0  scared to death
4  1   Abdominal rash


In [14]:
# Creating DataBunch for language modelling
data_lm = TextLMDataBunch.from_csv(path, fine_tuning_data)

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [17]:
data_clas.add_test(test,label=0)

In [0]:
data_lm.save()
data_clas.save()

In [19]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'the',
 'of',
 'and',
 'in',
 'to',
 'a',
 'for',
 'with',
 'diclofenac',
 'was',
 'were',
 'on',
 'is',
 'rt',
 'sodium',
 'by',
 'that',
 'as',
 'at',
 'patients',
 'or',
 'this',
 'from',
 'drug',
 'you',
 'i',
 'mg',
 'group',
 'be',
 'study',
 'disease',
 'treatment',
 'are',
 'after',
 'pain',
 'not',
 'an',
 'amp',
 'it',
 'have',
 'release',
 'p',
 'gluten',
 'we',
 'than',
 'free',
 'compared',
 'all',
 'no',
 'effect',
 'new',
 'more',
 'using',
 'effects',
 'groups',
 'both',
 'two',
 'our',
 'your',
 'about',
 'its',
 'h',
 'significantly',
 'inflammatory',
 'out',
 'significant',
 'time',
 'my',
 'between',
 'use',
 'these',
 'can',
 'but',
 'anti',
 'day',
 'drugs',
 'which',
 'voltaren',
 'potassium',
 'used',
 'dose',
 'has',
 'do',
 'also',
 'results',
 'placebo',
 'glutenfree',
 'one',
 's',
 'what',
 'may',
 'nt',
 'efficacy',
 'm',
 'there',
 'celiac',
 'ml',
 'treated',
 'day

In [20]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [21]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.189795,4.622593,0.252232,22:13


In [23]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.439962,4.277723,0.285882,22:28


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.124549,4.239039,0.290718,22:11


In [25]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.072823,4.208352,0.293424,22:15


In [0]:
learn.save_encoder('tuning_pubmed_bigtweet_overlap_0_first')

In [27]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('tuning_pubmed_bigtweet_overlap_0_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.692076,3.475826,0.442604,02:53


In [28]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.777728,2.973190,0.518343,02:53


In [29]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.300978,2.806721,0.533728,03:23


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.081958,2.553745,0.564497,03:33
1,1.893933,2.441428,0.589349,03:38


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.762751,2.331722,0.596450,03:16
1,1.634928,2.243957,0.620118,03:29


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.567878,2.138001,0.633136,03:31
1,1.475230,2.102811,0.628402,03:31


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.372947,2.081290,0.629586,03:42
1,1.331616,2.023888,0.643787,03:20


In [34]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.279974,2.039060,0.648521,03:30
1,1.252693,1.971475,0.659172,03:31


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.176934,1.969709,0.646154,03:16
1,1.184779,1.951771,0.648521,03:39
2,1.085015,1.891574,0.666272,03:21
3,1.002920,1.877904,0.666272,03:39


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.037601,1.882386,0.671006,03:48
1,1.056233,1.966191,0.656805,03:37
2,0.988894,1.854243,0.668639,03:15
3,0.933588,1.863005,0.678106,03:15


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.924321,1.891535,0.672189,03:27
1,0.977580,1.895770,0.669823,03:14
2,0.936676,1.889676,0.679290,03:24
3,0.889455,1.881060,0.689941,03:22


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.886217,1.855683,0.689941,03:40
1,0.940503,1.912513,0.678106,03:31
2,0.850190,1.867010,0.694675,03:26
3,0.839185,1.867149,0.689941,03:18


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.878209,1.927719,0.688757,03:48
1,0.918637,1.951802,0.680473,03:21
2,0.873383,1.967481,0.682840,03:20
3,0.799547,1.888378,0.691124,03:43


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.833247,1.923309,0.685207,03:47
1,0.854815,1.927199,0.680473,03:25
2,0.837687,1.892840,0.691124,03:24
3,0.797813,1.910377,0.692308,03:41


In [41]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.822142,1.961573,0.682840,03:41
1,0.874002,1.942618,0.686391,03:17
2,0.840978,1.914883,0.693491,03:25
3,0.813525,1.922315,0.693491,03:29


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.782283,1.977603,0.680473,03:39
1,0.816146,1.936542,0.682840,03:37


In [43]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.872658,1.974546,0.684024,03:39
1,0.788341,1.939488,0.691124,03:28


In [44]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.790506,1.917258,0.687574,03:25


In [0]:
# save the best model

learn.save_encoder('tuning_pubmed_bigtweet_overlap_0_last')

# Part three: Predict on the test dataset

In [46]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [47]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

1018
1018
[840, 623, 62, 334, 5, 1026, 325, 8, 8, 8, 8, 8, 8, 760, 10, 640, 5, 32, 418, 589, 27, 537, 27, 754, 289, 30, 30, 31, 31, 32, 209, 309, 309, 309, 37, 256, 623, 289, 603, 325, 325, 49, 49, 557, 49, 325, 49, 1035, 1035, 591, 100, 100, 100, 100, 245, 53, 53, 61, 281, 71, 71, 71, 491, 76, 76, 76, 260, 190, 418, 418, 418, 190, 309, 309, 309, 309, 309, 309, 309, 309, 309, 309, 309, 309, 309, 309, 309, 309, 309, 309, 309, 309, 309, 309, 309, 309, 289, 289, 289, 418, 309, 309, 80, 80, 80, 80, 80, 80, 80, 80, 425, 660, 89, 137, 627, 325, 91, 92, 517, 94, 826, 826, 94, 94, 94, 94, 325, 318, 98, 492, 98, 49, 644, 100, 337, 337, 100, 100, 100, 100, 694, 100, 291, 265, 805, 660, 660, 108, 209, 1035, 289, 111, 111, 111, 111, 265, 111, 111, 289, 289, 111, 227, 425, 120, 1026, 1026, 418, 125, 125, 125, 125, 125, 125, 125, 125, 418, 125, 127, 343, 289, 129, 289, 469, 180, 142, 142, 144, 285, 415, 418, 847, 2, 344, 157, 158, 194, 165, 359, 166, 166, 168, 298, 289, 265, 864, 864, 864, 864, 864,

In [48]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

458
0.449901768172888
